<a href="https://colab.research.google.com/github/matejkvassay/colab-notebooks/blob/master/langchain1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --upgrade -q pip openai langchain huggingface

In [18]:
from google.colab import userdata
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain.globals import set_debug, set_verbose

In [38]:
API_KEY = userdata.get('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=API_KEY, verbose=True)

# Basic prompt

In [4]:
prompt = ChatPromptTemplate.from_template("If {country} is a country tell me it's population. Otherwise politely refuse to answer.")
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"country": "bratislava"})

"I'm sorry, but Bratislava is not a country. It is the capital city of Slovakia."

# Basic prompt - output to JSON

In [50]:
class CityInfo(BaseModel):
    population: int = Field(description="population")
    explanation: str = Field(description="detailed step-by-step explanation of how was the population count derived")

parser = JsonOutputParser(pydantic_object=CityInfo)

TEMP = "Answer the user query.\n{format_instructions}\n. Query: Tell me population of {country}. Only answer if {country} is a country."


prompt = PromptTemplate(
    template=TEMP,
    input_variables=["country"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"country": "Paris"})

{'population': None, 'explanation': None}